In [1]:
# imports


import os
import requests
import json

In [2]:
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI

In [4]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# API 키 확인
if api_key and api_key.startswith("sk-proj") and len(api_key) >10:
    print('API key looks good so far')
else:
    print("API키에 문제가 있는 것 같습니다.")



API key looks good so far


In [ ]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()
# openai API model 설정

In [7]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


In [8]:
class Website:

    def __init__(self, url):
        self.url = url

        response = requests.get(url, headers=headers)

        self.body = response.content #response의 내용을 할당

        soup = BeautifulSoup(self.body, 'html.parser')
        
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", 'style', 'img', 'input']):
                irrelevant.decompose()
                # .decompose(): Beautiful Soup의 Tag 객체가 제공하는 메서드로, 
                # 해당 태그를 문서 트리에서 완전히 제거합니다. 
                # 제거된 태그와 그 하위 요소들은 더 이상 soup 객체에서 접근할 수 없게 됩니다.

        # soup : Beautiful Soup으로 파싱된 HTML 또는 XML 문서 전체를 나타내는 객체입니다.

        # .body: soup 객체의 속성 중 하나로, HTML 문서의 <body>태그와 그 내용을 담고 있는Tag객체를 반환합니다. 
        #         만약 HTML 문서에&lt;body> 태그가 없다면 None을 반환할 수 있습니다.

            self.text = soup.body.get_text(separator="\n", strip=True)

        else:
            self.text = "No body found"
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_content(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [9]:
lge = Website("https://www.lge.co.kr/")
lge.links

['/',
 '#none',
 '/my-page',
 '/company/main',
 '/business',
 '#none',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 'https://www.lge.co.kr/tvs',
 'https://www.lge.co.kr/tvs?subCateId=CT50000028',
 'https://www.lge.co.kr/tvs?subCateId=CT50041000',
 'https://www.lge.co.kr/tvs?subCateId=CT50000029',
 'https://www.lge.co.kr/tvs?subCateId=CT50000030',
 'https://www.lge.co.kr/tvs?subCateId=CT50000026',
 'https://www.lge.co.kr/tvs?subCateId=CT50029007',
 'https://www.lge.co.kr/tvs?subCateId=CT50220000',
 'https://www.lge.co.kr/stan-by-me',
 'https://www.lge.co.kr/projectors',
 'https://www.lge.co.kr/projectors?subCateId=CT50000033',
 'https://www.lge.co.kr/projectors?subCateId=CT50256007',
 'https://www.lge.co.kr/projectors?subCateId=CT50308000',
 'https://www.lge.co.kr/projectors?subCateId=CT50077001',
 'https://www.lge.co.kr/home-audio',
 'https://www.lge.co.kr/home-audio?subCateId=CT50000044',
 'https://www.lge.co.kr/home-audio?subCateId=CT50000040',
 'https://www.lge

In [10]:
link_system_prompt = "당신에게 웹페이지에서 발견된 링크들의 목록을 전달할 것입니다.\
    당신은 어떤 링크를 회사에 관한 브로슈어에 포함하는 것이 가장 관련성이 높은지 결정할 수 있습니다.\
    링크들은 about 페이지 또는 회사 페이지 또는 경력/ 채용정보 등의 링크 입니다"

link_system_prompt += "당신은 아래의 예시와 같이 JSON 형식으로 답변해야 합니다"
link_system_prompt += """
    {
        "links": [
            {"type": "about page", "url":"https://full.url/about"}
            {"type": "company page", "url":"'https://another.full.url/company"}
            ]
    }
    """

In [11]:
print(link_system_prompt)

당신에게 웹페이지에서 발견된 링크들의 목록을 전달할 것입니다.    당신은 어떤 링크를 회사에 관한 브로슈어에 포함하는 것이 가장 관련성이 높은지 결정할 수 있습니다.    링크들은 about 페이지 또는 회사 페이지 또는 경력/ 채용정보 등의 링크 입니다당신은 아래의 예시와 같이 JSON 형식으로 답변해야 합니다
    {
        "links": [
            {"type": "about page", "url":"https://full.url/about"}
            {"type": "company page", "url":"'https://another.full.url/company"}
            ]
    }
    


In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"여기에 웹사이트({website.url})의 링크 목록이 있습니다."
    user_prompt += f"링크들은 다음과 같습니다: {website.links}\n"

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )